In [1]:
!pip install torch

  Using cached torch-2.0.1-cp38-cp38-win_amd64.whl (172.4 MB)



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas 
!pip install numpy
!pip install seaborn
!pip install matplotlib

  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/293.3 kB ? eta -:--:--
     -------------------------------------- 293.3/293.3 kB 8.8 MB/s eta 0:00:00
  Using cached matplotlib-3.7.2-cp38-cp38-win_amd64.whl (7.5 MB)
  Using cached contourpy-1.1.0-cp38-cp38-win_amd64.whl (470 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.42.0-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached kiwisolver-1.4.4-cp38-cp38-win_amd64.whl (55 kB)
  Using cached Pillow-10.0.0-cp38-cp38-win_amd64.whl (2.5 MB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("names.txt",header=None)

In [8]:
df

,0
0,emma
1,olivia
2,ava
3,isabella
4,sophia
...,...
32028,zylas
32029,zyran
32030,zyrie
32031,zyron


NameError: name 'l' is not defined